In [28]:
import pandas as pd
import numpy as np
import os
import re

In [105]:
#import artsy data 
cur_path = os.getcwd()
result_path = 'ArtsyResult'
file_name = 'ArtsyData.csv'
path = os.path.join(cur_path,result_path,file_name)
df = pd.read_csv(path)
# drop all incomplete 
df = df.dropna(axis=0)
df = df.reset_index(drop=True)
df.tail()

,id,media,url,artist_name,artist_url,work_name,work_year,media_long,dimension,gallery_name,gallery_url,bid,price,img_url
179,design26,design,https://www.artsy.net/artwork/dale-chihuly-unt...,Dale Chihuly,https://www.artsy.net/artist/dale-chihuly,Untitled,1981,Hand Blown Glass,7 × 11 × 7 1/2 in | 17.8 × 27.9 × 19.1 cm,Original Art Broker,https://www.artsy.net/partner/original-art-broker,False,"US$8,995",https://d7hftxdivxxvm.cloudfront.net?resize_to...
180,design27,design,https://www.artsy.net/artwork/the-haas-brother...,The Haas Brothers,https://www.artsy.net/artist/the-haas-brothers,Animal Planet,2014,"Wool, Silk",144 in diameter | 365.8 cm diameter,Jeff Lincoln Art+Design,https://www.artsy.net/partner/jeff-lincoln-art...,False,"US$65,000–US$85,000",https://d7hftxdivxxvm.cloudfront.net?resize_to...
181,design28,design,https://www.artsy.net/artwork/marcel-breuer-ea...,Marcel Breuer,https://www.artsy.net/artist/marcel-breuer,"Early Canvas Model B3 ""Wassily"" Chair, Black E...",1927,Chromium-plated tubular steel and original Eis...,28 1/4 × 30 3/4 × 28 in | 71.8 × 78.1 × 71.1 cm,Forsyth,https://www.artsy.net/partner/forsyth,False,"US$30,000",https://d7hftxdivxxvm.cloudfront.net?resize_to...
182,design29,design,https://www.artsy.net/artwork/nina-murashkina-...,Nina Murashkina,https://www.artsy.net/artist/nina-murashkina,Hero,2022,"Ceramic, underglaze drawing, gold leaf, third",33 1/10 × 15 × 15 in | 84 × 38 × 38 cm,Lysenko MyGallery,https://www.artsy.net/partner/lysenko-mygallery,False,"£12,000",https://d7hftxdivxxvm.cloudfront.net?resize_to...
183,design30,design,https://www.artsy.net/artwork/facture-studio-d...,Facture Studio,https://www.artsy.net/artist/facture-studio,Drum Side Table,2022,"Resin, Wood",16 × 16 × 17 1/2 in | 40.6 × 40.6 × 44.5 cm,Tuleste Factory,https://www.artsy.net/partner/tuleste-factory,False,"US$4,500",https://d7hftxdivxxvm.cloudfront.net?resize_to...


In [106]:
# To change work_year to numeric

for i in range(len(df['work_year'])):
    if not df['work_year'][i].isnumeric():
        df['work_year'][i] = df['work_year'][i][-4:]
df['work_year'] = df['work_year'].astype(int)

/var/folders/2p/2p_g91q95gj9px5nbxvkbw3h0000gn/T/ipykernel_74899/4207513392.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['work_year'][i] = df['work_year'][i][-4:]


In [107]:
usd, pound, eur = "$", "£", "€"
#helpers
def toUsd(currency, amount):
    if currency == "pound":
        return amount*1.21
    if currency == "eur":
        return amount*1.04
    else:
        return amount

def getPrice (price):
    if not isinstance(price, str):
        return price
    if '.' in price:
        print("error:", price)
    if pound in price:
        currency = "pound"
    if eur in price:
        currency = "eur"
    else:
        currency = "usd"
    price_list = price.split("–")
    if len(price_list) > 2:
        print("error: ",price_list)
        return
    if len(price_list) == 1:
        amount =  ''.join(c for c in price if c.isdigit())
        return toUsd(currency,int(amount))
    else:
        amount1 =  ''.join(c for c in price_list[0] if c.isdigit())
        amount2 =  ''.join(c for c in price_list[1] if c.isdigit())
        amount = (int(amount1)+ int(amount2))//2
        return toUsd(currency,amount)
    


In [108]:
# observation: all prices are in unit usd, gbp or eur
usd, pound, eur = "$", "£", "€"
# To change price to number; estimated price to an actual number
count = 0
for i in range(len(df['price'])):
    df['price'][i] = getPrice(df['price'][i])
print(df['price'])

    



0      65000
1       2500
2       9600
3        850
4       5000
       ...  
179     8995
180    75000
181    30000
182    12000
183     4500
Name: price, Length: 184, dtype: object


/var/folders/2p/2p_g91q95gj9px5nbxvkbw3h0000gn/T/ipykernel_74899/2894904650.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'][i] = getPrice(df['price'][i])


In [109]:
df

,id,media,url,artist_name,artist_url,work_name,work_year,media_long,dimension,gallery_name,gallery_url,bid,price,img_url
0,painting1,painting,https://www.artsy.net/artwork/yaacov-agam-love...,Yaacov Agam,https://www.artsy.net/artist/yaacov-agam,LOVE STAR OF DAVID,1978,GOUACHE ON PAPER,29 × 22 1/5 in | 73.7 × 56.4 cm,Gallery Art,https://www.artsy.net/partner/gallery-art,False,65000,https://d7hftxdivxxvm.cloudfront.net?resize_to...
1,painting2,painting,https://www.artsy.net/artwork/corey-mason-blue...,Corey Mason,https://www.artsy.net/artist/corey-mason,Blue 200807,2021,Silkscreen ink on Rives BFK Paper with embosse...,30 × 22 in | 76.2 × 55.9 cm,C O U N T Y,https://www.artsy.net/partner/c-o-u-n-t-y,False,2500,https://d7hftxdivxxvm.cloudfront.net?resize_to...
2,painting3,painting,https://www.artsy.net/artwork/corey-mason-unti...,Corey Mason,https://www.artsy.net/artist/corey-mason,Untitled,2019,Pigment on reverse primed canvas,50 × 40 in | 127 × 101.6 cm,C O U N T Y,https://www.artsy.net/partner/c-o-u-n-t-y,False,9600,https://d7hftxdivxxvm.cloudfront.net?resize_to...
3,painting4,painting,https://www.artsy.net/artwork/anico-mostert-we...,Anico Mostert,https://www.artsy.net/artist/anico-mostert,We are best friends,2022,Oil on canvas,9 1/10 × 10 3/5 in | 23 × 27 cm,JO-HS,https://www.artsy.net/partner/jo-hs,False,850,https://d7hftxdivxxvm.cloudfront.net?resize_to...
4,painting5,painting,https://www.artsy.net/artwork/corey-mason-blac...,Corey Mason,https://www.artsy.net/artist/corey-mason,Black 3200701,2020,Silkscreen ink on linen,31 × 21 in | 78.7 × 53.3 cm,C O U N T Y,https://www.artsy.net/partner/c-o-u-n-t-y,False,5000,https://d7hftxdivxxvm.cloudfront.net?resize_to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,design26,design,https://www.artsy.net/artwork/dale-chihuly-unt...,Dale Chihuly,https://www.artsy.net/artist/dale-chihuly,Untitled,1981,Hand Blown Glass,7 × 11 × 7 1/2 in | 17.8 × 27.9 × 19.1 cm,Original Art Broker,https://www.artsy.net/partner/original-art-broker,False,8995,https://d7hftxdivxxvm.cloudfront.net?resize_to...
180,design27,design,https://www.artsy.net/artwork/the-haas-brother...,The Haas Brothers,https://www.artsy.net/artist/the-haas-brothers,Animal Planet,2014,"Wool, Silk",144 in diameter | 365.8 cm diameter,Jeff Lincoln Art+Design,https://www.artsy.net/partner/jeff-lincoln-art...,False,75000,https://d7hftxdivxxvm.cloudfront.net?resize_to...
181,design28,design,https://www.artsy.net/artwork/marcel-breuer-ea...,Marcel Breuer,https://www.artsy.net/artist/marcel-breuer,"Early Canvas Model B3 ""Wassily"" Chair, Black E...",1927,Chromium-plated tubular steel and original Eis...,28 1/4 × 30 3/4 × 28 in | 71.8 × 78.1 × 71.1 cm,Forsyth,https://www.artsy.net/partner/forsyth,False,30000,https://d7hftxdivxxvm.cloudfront.net?resize_to...
182,design29,design,https://www.artsy.net/artwork/nina-murashkina-...,Nina Murashkina,https://www.artsy.net/artist/nina-murashkina,Hero,2022,"Ceramic, underglaze drawing, gold leaf, third",33 1/10 × 15 × 15 in | 84 × 38 × 38 cm,Lysenko MyGallery,https://www.artsy.net/partner/lysenko-mygallery,False,12000,https://d7hftxdivxxvm.cloudfront.net?resize_to...



### If needed, will change the format for dimension to numeric

plan:

will replace the dimension column to 3 columns (length, wide, height), unit: cm
will add one column "three_d" type: boolean, to indicate if it is a 3 dimension work
height will be 0 for non 3 dimensional works

Raw data format examples: 
16 × 16 × 17 1/2 in | 40.6 × 40.6 × 44.5 cm
144 in diameter | 365.8 cm diameter
7 × 11 × 7 1/2 in | 17.8 × 27.9 × 19.1 cm